In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import namedtuple
import vega_sim
from vega_sim.null_service import VegaServiceNull

In [3]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "pin")

AUCTION1 = WalletConfig("auction1", "auction1")
AUCTION2 = WalletConfig("auction2", "auction2")

TRADER_WALLET = WalletConfig("TRADER", "TRADER")

RANDOM_WALLET = WalletConfig("RANDOM", "RANDOM")

TERMINATE_WALLET = WalletConfig("TERMINATE", "TERMINATE")

WALLETS = [MM_WALLET, TRADER_WALLET, RANDOM_WALLET, TERMINATE_WALLET, AUCTION1, AUCTION2]


TRADER1_WALLET = WalletConfig("TRADER1", "TRADER")

TRADER2_WALLET = WalletConfig("TRADER2", "TRADER")

In [4]:
vega = VegaServiceNull(warn_on_raw_data_access=False,run_with_console=False, transactions_per_block=1, retain_log_files=True)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/yj/cjhtlxn90wldd1hvw5lkxnrc0000gn/T/tmps83lipaq
INFO:vega_sim.null_service:Launching GraphQL node at port 53553
INFO:vega_sim.null_service:Launching Console at port 53559


In [5]:
vega.wallet_url

'http://localhost:53550'

In [6]:
for wallet in WALLETS + [TRADER1_WALLET, TRADER2_WALLET]:
    vega.create_wallet(wallet.name, wallet.passphrase)

In [7]:
vega.mint(MM_WALLET.name, asset="VOTE", amount=1e4,)
vega.forward("10s")
vega.wait_for_total_catchup()

In [8]:
# Create asset
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=1e10,
)
vega.wait_fn(10)
vega.wait_for_datanode_sync()

In [9]:
tDAI_id = vega.find_asset_id(symbol="tDAI")

In [10]:
for wallet in WALLETS:
    vega.mint(
        wallet.name,
        asset=tDAI_id,
        amount=1000000,
    )
vega.forward("10s")
vega.wait_for_total_catchup()

In [11]:
vega.mint(
        TRADER1_WALLET.name,
        asset=tDAI_id,
        amount=2000,
    )
vega.mint(
        TRADER2_WALLET.name,
        asset=tDAI_id,
        amount=90,
    )

In [12]:
vega.party_account(MM_WALLET.name, tDAI_id, None)

AccountData(general=1000000.0, margin=0, bond=0)

In [13]:
vega.create_simple_market(
    market_name="CRYPTO:BTCDAI/DEC22",
    proposal_wallet=MM_WALLET.name,
    settlement_asset_id=tDAI_id,
    termination_wallet=TERMINATE_WALLET.name,
    market_decimals=5,
)
market_id = vega.all_markets()[0].id
vega.submit_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=1000,
    fee=0.002,
    buy_specs=[("PEGGED_REFERENCE_BEST_BID", 2, 1)],
    sell_specs=[("PEGGED_REFERENCE_BEST_ASK", 2, 1)],
    is_amendment=False,
)

In [14]:
# Add transactions in the proposed market to pass opening auction at price 0.3
vega.submit_order(
    trading_wallet=AUCTION1.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=10,
    price=50,
)

vega.submit_order(
    trading_wallet=AUCTION2.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=10,
    price=50,
)

vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=25,
    wait=True,
)
vega.submit_order(
    trading_wallet=RANDOM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=75,
    wait=True,
)

'ca09da5341ddbe3950f686441fe866760b351d9f2694794e9061c5e040966ff2'

In [15]:
vega.market_data(market_id)

mark_price: "5000000"
best_bid_price: "2500000"
best_bid_volume: 81
best_offer_price: "7500000"
best_offer_volume: 28
best_static_bid_price: "2500000"
best_static_bid_volume: 1
best_static_offer_price: "7500000"
best_static_offer_volume: 1
mid_price: "5000000"
static_mid_price: "5000000"
market: "80d6b36aa98dfde4246305954a4cea863060e2cac83dc8d887171ce67b2670c1"
timestamp: 1638185358259066000
open_interest: 10
indicative_price: "0"
market_trading_mode: TRADING_MODE_CONTINUOUS
target_stake: "2710500"
supplied_stake: "100000000"
price_monitoring_bounds {
  min_valid_price: "3865599"
  max_valid_price: "6449621"
  trigger {
    horizon: 86400
    probability: "0.999999"
    auction_extension: 5
  }
  reference_price: "5000000"
}
market_value_proxy: "100000000"
liquidity_provider_fee_share {
  party: "d7847dba7b383cc7842b40b9533cbbfed520a01b2e7369bc63987f8810d8baea"
  equity_like_share: "1"
  average_entry_valuation: "100000000"
}
market_state: STATE_ACTIVE

In [16]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 81.0}, asks={75.0: 27.0})

In [17]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.stakeToCcySiskas", "0.1")

### This should probably have changed

In [18]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 81.0}, asks={75.0: 27.0})

In [19]:
vega.forward('10s')

In [20]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 81.0}, asks={75.0: 27.0})

### But actually only changes once we place an order

In [21]:
order_id = vega.submit_order(
    trading_wallet=TRADER1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=50,
    wait = True
)
vega.cancel_order(TRADER1_WALLET.name, market_id, order_id)

In [22]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 9.0}, asks={75.0: 3.0})

### Let's check on the MM

In [23]:
vega.party_account(MM_WALLET.name, tDAI_id, market_id)

AccountData(general=998975.26492, margin=24.73508, bond=1000.0)

### Now raise and prod again

In [24]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.stakeToCcySiskas", "1")

In [25]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 9.0}, asks={75.0: 3.0})

In [26]:
order_id = vega.submit_order(
    trading_wallet=TRADER1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=50,
    wait = True
)
vega.cancel_order(TRADER1_WALLET.name, market_id, order_id)

In [27]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 81.0}, asks={75.0: 27.0})

### Now increase a little and prod

In [28]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.stakeToCcySiskas", "1.1")

In [29]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 81.0}, asks={75.0: 27.0})

In [30]:
order_id = vega.submit_order(
    trading_wallet=TRADER1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=50,
    wait = True
)
vega.cancel_order(TRADER1_WALLET.name, market_id, order_id)

In [31]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 89.0}, asks={75.0: 30.0})

### And now back down

In [32]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.stakeToCcySiskas", "1")

In [33]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 89.0}, asks={75.0: 30.0})

In [34]:
order_id = vega.submit_order(
    trading_wallet=TRADER1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=50,
    wait = True
)
vega.cancel_order(TRADER1_WALLET.name, market_id, order_id)

In [35]:
vega.order_book_by_market(market_id)

OrderBook(bids={25.0: 1.0}, asks={75.0: 1.0})

### Uhhh....

In [36]:
vega.party_account(MM_WALLET.name, tDAI_id, market_id)

AccountData(general=0.0, margin=724979.09662, bond=0.0)

In [37]:
vega.positions_by_market(MM_WALLET.name, market_id)

[]

In [40]:
vega.positions_by_market(TRADER1_WALLET.name, market_id)

[]

In [38]:
vega.market_data(market_id)

mark_price: "5000000"
best_bid_price: "2500000"
best_bid_volume: 1
best_offer_price: "7500000"
best_offer_volume: 1
best_static_bid_price: "2500000"
best_static_bid_volume: 1
best_static_offer_price: "7500000"
best_static_offer_volume: 1
mid_price: "5000000"
static_mid_price: "5000000"
market: "80d6b36aa98dfde4246305954a4cea863060e2cac83dc8d887171ce67b2670c1"
timestamp: 1638185866259066000
open_interest: 10
auction_end: 1638185866259066000
auction_start: 1638185863259066000
indicative_price: "0"
market_trading_mode: TRADING_MODE_MONITORING_AUCTION
trigger: AUCTION_TRIGGER_LIQUIDITY
extension_trigger: AUCTION_TRIGGER_LIQUIDITY
target_stake: "2710500"
supplied_stake: "0"
price_monitoring_bounds {
  min_valid_price: "3865599"
  max_valid_price: "6449621"
  trigger {
    horizon: 86400
    probability: "0.999999"
    auction_extension: 5
  }
  reference_price: "5000000"
}
market_value_proxy: "0"
market_state: STATE_SUSPENDED

In [39]:
# vega.stop()